### Import library

In [76]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [78]:
toronto_df = pd.read_csv('toronto_df.csv')
toronto_df.drop(['Unnamed: 0'],axis=1,inplace=True)
#toronto_df.head(5)
toronto_df.shape

(77, 5)

### 1.  Map Toronto

In [18]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['latitude'], toronto_df['longtitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

#map_toronto

###  2. Define Foursquare Credentials and Version
with some parameters

In [82]:
CLIENT_ID = 'DBMBAOKP15B3BFT0T4O4KODS4HU2IGKTUE0OESSHM0Q15TC1' # your Foursquare ID
CLIENT_SECRET = 'QGZTNMNUIAIOFSPHLM0EJ5G4WTVKLSMB2WGNT54ORI5H0LIR' # your Foursquare Secret
VERSION = '20200210' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBMBAOKP15B3BFT0T4O4KODS4HU2IGKTUE0OESSHM0Q15TC1
CLIENT_SECRET:QGZTNMNUIAIOFSPHLM0EJ5G4WTVKLSMB2WGNT54ORI5H0LIR


### 3. Now, let's get the top 100 venues that are in 'Upper Rouge', from Scarborough 
 within a radius of 500 meters.

In [101]:
# filter and reset index
scarborough_data = toronto_df[toronto_df['Borough'] == 'Scarborough'].reset_index(drop=True)

neighborhood_latitude = scarborough_data.loc[10, 'latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[10, 'longtitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[10, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
scarborough_data.head()

Latitude and longitude values of "Maryvale, Wexford" are 43.7500715, -79.2958491.


,PostCode,Borough,Neighbourhood,latitude,longtitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476


In [91]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL

### 3.2 Send the GET request and examine the resutls

In [102]:
results = requests.get(url).json()
#results

In [93]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/aestheticism/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Crown Pastries,Bakery,43.746098,-79.293142
1,Wexford Restaurant,Breakfast Spot,43.746030,-79.293843
2,Subway,Sandwich Place,43.746267,-79.293193
3,Lebanese bakery,Middle Eastern Restaurant,43.746701,-79.292896
4,Wexford Heights Plaza,Shopping Mall,43.746136,-79.293782


In [94]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


### 4. Explore Neighborhoods in Manhattan

In [104]:
# get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [105]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['latitude'],
                                   longitudes=scarborough_data['longtitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Morningside, West Hill
Woburn
Scarborough Village
Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside
Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Tam O'Shanter
Agincourt North, Milliken
Upper Rouge


In [111]:
# scarborough_venues.head()
print(scarborough_venues.shape)
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))
scarborough_venues.groupby('Neighborhood').count()

(73, 7)
There are 52 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"Ionview, Kennedy Park",5,5,5,5,5,5
"Maryvale, Wexford",8,8,8,8,8,8


In [113]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [124]:
scarborough_onehot.shape

(73, 53)

In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [130]:
scarborough_venues_grouped =scarborough_onehot.groupby('Neighborhood').mean().reset_index()

#give a num
num = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_venues_grouped['Neighborhood']

for ind in np.arange(scarborough_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_venues_grouped.iloc[ind, :], num)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Vietnamese Restaurant,Convenience Store,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint
1,"Agincourt North, Milliken",Park,Playground,Clothing Store,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Park,Convenience Store,General Entertainment
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Movie Theater,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Bank,Bar,Ice Cream Shop,General Entertainment,American Restaurant
6,"Highland Creek, Rouge Hill, Port Union",Bar,Moving Target,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,"Ionview, Kennedy Park",Coffee Shop,Bus Station,Discount Store,Chinese Restaurant,Department Store,College Stadium,Indian Restaurant,Ice Cream Shop,General Entertainment,Gas Station
8,"Maryvale, Wexford",Middle Eastern Restaurant,Accessories Store,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,College Stadium,General Entertainment,Gas Station
9,"Morningside, West Hill",Spa,Intersection,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium


## Cluster Groups

In [135]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(13)
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_venues_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 0, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int32)

In [136]:
# label groups
scarborough_merged = scarb_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged

,PostCode,Borough,Neighbourhood,latitude,longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711,1,Spa,Intersection,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,American Restaurant,Auto Garage,Indian Restaurant,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,1,Coffee Shop,Bus Station,Discount Store,Chinese Restaurant,Department Store,College Stadium,Indian Restaurant,Ice Cream Shop,General Entertainment,Gas Station
6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,3,Bakery,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Park,Convenience Store,General Entertainment
7,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,1,American Restaurant,Motel,Movie Theater,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
9,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1,Indian Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Bank,Bar,Ice Cream Shop,General Entertainment,American Restaurant


In [143]:
address_scarborough = 'Scarborough,Toronto'
latitude_scarborough = 43.773077
longtitude_scarborough = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scarborough, longtitude_scarborough))

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


map_scarborough= folium.Map(location=[latitude_scarborough, longtitude_scarborough], zoom_start=12)

# create map
map_clusters = folium.Map(location = [latitude_scarborough, longtitude_scarborough], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['latitude'], scarborough_merged['longtitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


## Show clusters distinctively

In [144]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,Scarborough,0,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Vietnamese Restaurant,Convenience Store,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint


In [145]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
2,Scarborough,1,Spa,Intersection,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium
3,Scarborough,1,Coffee Shop,Korean Restaurant,American Restaurant,Auto Garage,Indian Restaurant,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,1,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,Scarborough,1,Coffee Shop,Bus Station,Discount Store,Chinese Restaurant,Department Store,College Stadium,Indian Restaurant,Ice Cream Shop,General Entertainment,Gas Station
7,Scarborough,1,American Restaurant,Motel,Movie Theater,Vietnamese Restaurant,College Stadium,Ice Cream Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
8,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
9,Scarborough,1,Indian Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Bank,Bar,Ice Cream Shop,General Entertainment,American Restaurant
12,Scarborough,1,Pizza Place,Italian Restaurant,Intersection,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Pharmacy


In [146]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,2,Middle Eastern Restaurant,Accessories Store,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,College Stadium,General Entertainment,Gas Station


In [147]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Bakery,Bus Line,Intersection,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Park,Convenience Store,General Entertainment


In [148]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
